[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/medsiglip/blob/main/using_medsiglip_model.ipynb)

Note: If using in colab, you need to install:

`pip install fiftyone huggingface-hub accelerate`

### ℹ️  Important! Be sure to request access to the model!

This is a gated model, so you will need to fill out the form on the model card: https://huggingface.co/google/medsiglip-448

Approval should be instantaneous.

You'll also have to set your Hugging Face in your enviornment:

```bash
export HF_TOKEN="your_token"
```

Or sign-in to Hugging Face via the CLI:

```bash
huggingface-cli login
```

# How to use MedSigLIP Model for Embeddings and Text Similarity Search

In [ ]:
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "Voxel51/SLAKE",
    name="SLAKE",
    overwrite=True,
    max_samples=10
    )

# Setup Zoo Model

In [ ]:
import fiftyone.zoo as foz

foz.register_zoo_model_source("https://github.com/harpreetsahota204/medsiglip", overwrite=True)

In [ ]:
foz.download_zoo_model(
    "https://github.com/harpreetsahota204/medsiglip",
    model_name="google/medsiglip-448",
)

In [ ]:
import fiftyone.zoo as foz

model = foz.load_zoo_model(
    "medsiglip-448"
    )

# Compute embeddings

In [ ]:
dataset.compute_embeddings(
    model=model,
    embeddings_field="medsiglip_embeddings",
)

# Compute visualization of embeddings

Note requires that `umap-learn` is installed. Currently, `umap-learn` only supports `numpy<=2.1.0`  

In [ ]:
import fiftyone.brain as fob

results = fob.compute_visualization(
    dataset,
    embeddings="medsiglip_embeddings",
    method="umap",
    brain_key="medsiglip_viz",
    num_dims=2,
)

# Build a similarity index for natural language search

You can [visit the docs](https://docs.voxel51.com/api/fiftyone.brain.html?highlight=compute_similarity#fiftyone.brain.compute_similarity) for more information on similarity search.

In [ ]:
import fiftyone.brain as fob

text_img_index = fob.compute_similarity(
    dataset,
    model="google/medsiglip-448", #or just pass in the already instantiated model
    brain_key="medsiglip_sim",
)

Verify that we can support text search:

In [ ]:
print(text_img_index.config.supports_prompts)  # True

In [ ]:
sims = text_img_index.sort_by_similarity(
    "healthy chest x-rays"
)

In [ ]:
sims

Select your Dataset from the dropdown menu, open the embeddings panel by clicking the `+` next to the Samples viewer, and select the embeddings you want to display by selecting from the dropdown menu in the embeddings panel.

To search via natural language in the App click the `🔎` button and type in your query. The most similar samples to the query will be shown in decreasing order of similarity

In [ ]:
fo.launch_app(dataset)